<a href="https://colab.research.google.com/github/Benish-Lezdo/ai_learnings/blob/main/colpoli_document_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ColPali Example



```
# This is formatted as code
```



In [ ]:
!pip install --upgrade byaldi
!pip install  poppler-utils
!pip install -q pdf2image git+https://github.com/huggingface/transformers.git qwen-vl-utils flash-attn




In [ ]:

!pip install -q pdf2image
!pip install flash-attn

!sudo apt-get install -y poppler-utils

from byaldi import RAGMultiModalModel
from huggingface_hub import notebook_login
from pdf2image import convert_from_path

images = convert_from_path("/content/pdf/injury_report.pdf")
images[0]

In [ ]:
import os
from byaldi import RAGMultiModalModel
from pathlib import Path
import torch
from torch.quantization import quantize_dynamic

# !wget https://arxiv.org/pdf/1706.03762
# !mkdir docs
# !mv 1706.03762 docs/attention.pdf
# !cp -r docs/attention.pdf docs/attention_with_a_mustache.pdf

# model = RAGMultiModalModel.from_pretrained("vidore/colpali")
model = RAGMultiModalModel.from_pretrained("vidore/colpali-v1.2")




index_name = "image_index"

model.index(
    input_path=Path("/content/pdf"),
    index_name= index_name,
    store_collection_with_index=False,
    overwrite=True,
)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Indexing file: /content/pdf/injury_report.pdf


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.


RuntimeError: Input type (torch.FloatTensor) and weight type (CPUBFloat16Type) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [12]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# Load the BioBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

# Function to embed a chunk of text
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Extract the [CLS] token embedding as a representation of the text

    print(outputs, "kkkkkkkkk")
    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings

# Function to split a document into sentences
def split_into_sentences(document):
    # Assuming the document is a string of text, split by periods or newlines.
    sentences = document.split('. ')
    return sentences

# Function to answer a query from a document
def answer_query(document, query):
    # Step 1: Split the document into sentences or chunks
    chunks = split_into_sentences(document)

    # Step 2: Embed the query
    query_embedding = embed_text(query)

    # Step 3: Embed each chunk of the document
    chunk_embeddings = [embed_text(chunk) for chunk in chunks]

    # Step 4: Calculate similarity between the query and each chunk
    similarities = [F.cosine_similarity(chunk_embedding, query_embedding, dim=1) for chunk_embedding in chunk_embeddings]

    # Step 5: Find the most similar chunk (highest similarity score)
    most_similar_chunk_idx = torch.argmax(torch.cat(similarities))

    # Step 6: Return the most relevant chunk of the document as the answer
    return chunks[most_similar_chunk_idx]

# Example usage
document = """
Cardiovascular disease is the leading cause of death worldwide. It affects millions of people every year.
There are several types of cardiovascular diseases, including coronary artery disease and heart failure.
Prevention includes lifestyle changes, such as regular exercise and a healthy diet.
"""

query = "What is the leading cause of death?"

# Get the most relevant sentence or chunk from the document
answer = answer_query(document, query)
print(f"Answer: {answer}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3766,  0.1432, -0.3651,  ..., -0.1829, -0.0042, -0.5757],
         [ 0.0701, -0.1438, -0.1996,  ...,  0.0226, -0.8468, -0.3335],
         [ 0.1158,  0.3318, -0.1784,  ...,  0.2432,  0.2813, -0.3980],
         ...,
         [-0.3711, -0.0717, -0.2687,  ...,  0.1181, -0.1134, -0.4047],
         [ 0.3919, -0.1858, -0.0445,  ...,  0.1552, -0.4503, -0.4781],
         [-0.2654,  0.2086, -0.6569,  ..., -0.0192, -0.2300, -0.9680]]]), pooler_output=tensor([[ 9.5044e-02, -3.5819e-02,  9.8310e-01, -1.0000e+00,  1.0000e+00,
          2.0829e-01, -6.9803e-02,  9.5632e-01, -1.1651e-01, -1.0391e-01,
          9.2999e-01,  9.9997e-01,  5.2889e-01, -9.2617e-01,  4.7640e-03,
          1.4112e-01,  1.0000e+00,  1.1433e-02, -9.9923e-01, -1.9318e-01,
         -1.1282e-01, -9.7692e-01, -2.0027e-01,  9.8232e-01,  3.2518e-02,
          1.3087e-01,  9.9995e-01,  9.9910e-01, -1.4612e-01,  8.3667e-02,
          3.6302e-02, -1.0000e+00,  